In [1]:
import pandas as pd
from datetime import timedelta


#### Reading data 
First, I read data from 12 csv files, corresponding to data from 12 months <br>
I change 2 fields started_at and ended_at to their corresponding format - datetime <br>
And then combine them into 1 dataframe called df 

In [2]:
csv_files = ['202207-divvy-tripdata.csv', '202208-divvy-tripdata.csv','202209-divvy-publictripdata.csv', 
             '202210-divvy-tripdata.csv', '202211-divvy-tripdata.csv', '202212-divvy-tripdata.csv', 
             '202301-divvy-tripdata.csv', '202302-divvy-tripdata.csv', '202303-divvy-tripdata.csv', 
             '202304-divvy-tripdata.csv', '202305-divvy-tripdata.csv', '202306-divvy-tripdata.csv']

dfs = []
for filename in csv_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df[['started_at', 'ended_at']] = df[['started_at', 'ended_at']].apply(pd.to_datetime)
    dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779379 entries, 0 to 5779378
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 573.2+ MB
None


#### Processing
To process data, I delete rows that have NaN values, duplicated rows and rows that have started time equals or larger than ended time

In [3]:
df = df.dropna()
df = df.drop_duplicates()
df = df.drop(df[df.started_at >= df.ended_at].index)

In [4]:
df['ride_length'] = df['ended_at'] - df['started_at']
df['start_weekday'] = df['started_at'].dt.day_name()
print(df)

                  ride_id  rideable_type          started_at  \
0        954144C2F67B1932   classic_bike 2022-07-05 08:12:47   
1        292E027607D218B6   classic_bike 2022-07-26 12:53:38   
2        57765852588AD6E0   classic_bike 2022-07-03 13:58:49   
3        B5B6BE44314590E6   classic_bike 2022-07-31 17:44:21   
4        A4C331F2A00E79E0   classic_bike 2022-07-13 19:49:06   
...                   ...            ...                 ...   
5779374  F5769BAFB81F2A68  electric_bike 2023-06-30 23:59:00   
5779375  CFC627752C368170  electric_bike 2023-06-30 23:59:00   
5779376  DA633467FDF61D11   classic_bike 2023-06-30 23:59:00   
5779377  A6814A2943D5CFC2  electric_bike 2023-06-30 23:59:00   
5779378  41FC14F87DA92588  electric_bike 2023-06-30 23:59:00   

                   ended_at          start_station_name start_station_id  \
0       2022-07-05 08:24:32  Ashland Ave & Blackhawk St            13224   
1       2022-07-26 12:55:31  Buckingham Fountain (Temp)            15541   
2  

#### Analyzing data
First, I divide df to 2 sub dataframe of member and casual customers to get some statistics like total rides, total ride length, average ride time of each dataset. 

In [5]:
member_df = df.loc[df['member_casual'] == 'member']
casual_df = df.loc[df['member_casual'] == 'casual']
print(member_df.ride_length.describe())
print(casual_df.ride_length.describe())
print(df.ride_length.describe())
print('member:', member_df['ride_length'].dt.total_seconds().sum())
print('casual:', casual_df['ride_length'].dt.total_seconds().sum())

count                      2729610
mean     0 days 00:12:07.083207491
std      0 days 00:19:35.891962222
min                0 days 00:00:01
25%                0 days 00:05:01
50%                0 days 00:08:43
75%                0 days 00:14:55
max                1 days 00:57:52
Name: ride_length, dtype: object
count                      1673272
mean     0 days 00:22:31.660317031
std      0 days 00:53:55.565913810
min                0 days 00:00:01
25%                0 days 00:07:17
50%                0 days 00:12:56
75%                0 days 00:24:02
max               22 days 05:55:27
Name: ride_length, dtype: object
count                      4402882
mean     0 days 00:16:04.447595007
std      0 days 00:36:59.850994962
min                0 days 00:00:01
25%                0 days 00:05:45
50%                0 days 00:10:00
75%                0 days 00:17:56
max               22 days 05:55:27
Name: ride_length, dtype: object
member: 1984653594.0
casual: 2261695362.0


Then I create a dataframe which querry month, type of bike, type of member, and calculate average ride time and count of each observation.<br>
And then save it to a excel sheet. 

In [6]:
df['month'] = df.loc[:,'started_at'].dt.month
month_count= df.groupby(['month','rideable_type','member_casual']).size()
month_count = month_count.reset_index(name='count')
avg_ride_len = df.groupby(['month','rideable_type','member_casual']).ride_length.mean().dt.total_seconds()
avg_ride_len = avg_ride_len.reset_index(name='average ride length(s)')

month_df = pd.merge(month_count, avg_ride_len, on = ['member_casual', 'rideable_type', 'month'])
#month_df['average ride length'] = month_df['average ride length'].apply(lambda x: str(x).split('.')[0])
#month_df['average ride length'] = month_df['average ride length'].apply(lambda x: x.strftime('%H:%M:%S'))
print(month_df)
with pd.ExcelWriter('cyclistics_data.xlsx', mode = 'a', if_sheet_exists='replace') as writer:
    month_df.to_excel(writer, sheet_name='bike_type_month', index=False)

    month  rideable_type member_casual   count  average ride length(s)
0       1   classic_bike        casual   13860             1033.742208
1       1   classic_bike        member   76359              640.316204
2       1    docked_bike        casual    1682             2307.323424
3       1  electric_bike        casual   14076              585.042057
4       1  electric_bike        member   42303              527.995178
5       2   classic_bike        casual   15446             1219.211446
6       2   classic_bike        member   74220              668.970951
7       2    docked_bike        casual    2151             2575.920037
8       2  electric_bike        casual   15177              683.988469
9       2  electric_bike        member   42558              549.281522
10      3   classic_bike        casual   19380             1217.272343
11      3   classic_bike        member   87595              670.434089
12      3    docked_bike        casual    2944             2514.290761
13    

Then I create a dataframe which querry hour, weekday, type of member, and calculate average ride time and count of each observation.<br>
And then save the data to a excel sheet.

In [7]:
df['hour'] = df.loc[:,'started_at'].dt.hour
customer_hour = df.groupby(['hour', 'start_weekday','member_casual']).size()
customer_hour = customer_hour.reset_index(name='count')
customer_hour['hour'] = customer_hour['hour'].astype(int)
avg_ride_len = df.groupby(['hour', 'start_weekday','member_casual']).ride_length.mean().dt.total_seconds()
avg_ride_len = avg_ride_len.reset_index(name='average ride length(s)')

customer_hour = pd.merge(customer_hour, avg_ride_len, on = ['hour', 'start_weekday','member_casual'])

print(customer_hour)
with pd.ExcelWriter('cyclistics_data.xlsx', mode='a', if_sheet_exists='replace') as writer:
    customer_hour.to_excel(writer, sheet_name='hour_weekday', index=False)

     hour start_weekday member_casual  count  average ride length(s)
0       0        Friday        casual   3624             1201.588852
1       0        Friday        member   3342              661.594853
2       0        Monday        casual   2399             1295.751146
3       0        Monday        member   2004              679.911178
4       0      Saturday        casual   8254             1163.906954
..    ...           ...           ...    ...                     ...
331    23      Thursday        member   6364              684.642206
332    23       Tuesday        casual   3267             1064.178145
333    23       Tuesday        member   4265              672.088159
334    23     Wednesday        casual   4301             1036.856312
335    23     Wednesday        member   5088              693.511399

[336 rows x 5 columns]


Then I find top 50 stations that have the most rides from casual customers, these stations may be good for promoting membership advertisement so casual customers can see membership upgrading plans and membership benefits.  

In [9]:
casual_station_count = casual_df.loc[:,'start_station_name'].value_counts()
casual_station_count = casual_station_count.nlargest(10)
casual_station_count = casual_station_count.reset_index(name='count')
casual_station_count = casual_station_count.rename(columns={'index': 'station'})

print(casual_station_count)
with pd.ExcelWriter('cyclistics_data.xlsx', mode='a', if_sheet_exists='replace') as writer:
    casual_station_count.to_excel(writer, sheet_name='top_casual_start_station', index=False)

                              station  count
0             Streeter Dr & Grand Ave  49711
1   DuSable Lake Shore Dr & Monroe St  29042
2               Michigan Ave & Oak St  22352
3                     Millennium Park  21927
4  DuSable Lake Shore Dr & North Blvd  20297
5                      Shedd Aquarium  18508
6                 Theater on the Lake  16093
7               Wells St & Concord Ln  13493
8                      Dusable Harbor  13191
9          Indiana Ave & Roosevelt Rd  11959
